# CCS+Q potentials with LAMMPS 

In the previous example, we have  shown the usage of CCS potential with pair_style table command with LAMMPS. This example demostrates the usage of CCS+Q potentials with LAMMPS. Here, we make use of the pair_style hybrid/overlay (https://docs.lammps.org/pair_hybrid.html). The goal is to use the efficient electrostatic implementation in LAMMPS to save computational time. In the original implementation of CCS+Q potential, the electrostatic contributions to the energy and forces are taken into account via ewald summation as implemented in PYMATGEN package.  

If consistent settings are not used there could be discrepancy between ewald energies computed via LAMMPS and PYMATGEN. Nevertheless, it is indeed possible for the user to construct a training-set with ewald energies computed by LAMMPS. However, this feature is not yet included in the current version of the code CCS code.  Additionally, one needs to be careful when using Ewald summation for surfaces ( surface corrections should be used) and non-periodic systems (direct method should be used not Ewald). 

In this example we  only consider bulk  ZnO systems for training CCS+Q potential. The ewald energies in the training set are computed via PYMATGEN as implemeneted in CCS code. Firstly, we check the consistency between ewald energies  obtained by LAMMPS and PYMATGEN. Therafter, we check the optimization of structure with CCS calculator as implemented in ASE and CCS tabular potential in LAMMPS.   


**Note:** 
* The LAMMPS must be complied with KSPACE package (https://docs.lammps.org/kspace_style.html) for ewald summation.
* Due to constraints in LAMMPS, real space cut-off in ewald summation should equal the table cut-off for short range.
* Install scikit-learn if it's not installed yet; 
https://scikit-learn.org/stable/install.html
* Install pymatgen if it's not installed yet;
https://pymatgen.org/installation.html#

# 1. Test for ewald energy from PYMATGEN and LAMMPS

In [1]:
import json
import copy
import numpy as np
import ase.db as db

from ase import Atoms
from pymatgen.core import Lattice, Structure
from pymatgen.analysis import ewald
from ase.calculators.lammpsrun import LAMMPS
from sklearn.metrics import mean_squared_error
from ase.io import read
from ase.io.trajectory import Trajectory
from ase.optimize import BFGS
from tqdm import tqdm

from ccs_fit.ase_calculator.ccs_ase_calculator import ew, CCS
from ccs_fit.scripts.jsonTotable import asecalcTotable
from ccs_fit.fitting.main import twp_fit as ccs_fit


In [2]:
q={"Zn":2.0,"O":-2.0}  # intial charges 

data = db.connect('DFT.db')

structures= list(data.select(limit=5))  # select 5 structures

pymat_ene=[]
pymat_force=[]


for config in tqdm(structures):
    struct = config.toatoms()
    ewald = ew(struct,q)
    pymat_ene.append(ewald.total_energy)
    pymat_force.append(np.ravel(ewald.forces))
    
print(" Ewald energy from PYMATGEN:{}".format(pymat_ene))

100%|█████████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.06s/it]

 Ewald energy from PYMATGEN:[-8170.758011429012, -8184.595515688825, -8186.09997270271, -8188.444604228178, -8213.698503604448]



In [3]:
# Parameters for LAMMPS 
parameters = {
              
              'atom_style': 'charge',                      # atomic system with charges so atom_style: charge     
              'pair_style': 'coul/long 5.99',              # Real space cut-off for ewald (close to PYMATGEN value but not exact)
              'pair_coeff': ['* *'],
              'kspace_style': 'ewald 0.000001',            # Precision for ewald calculation
              'command': '/usr/bin/lmp'                    # Remove or update based on location of your own lammps executable

}


lammps_ene=[]
lammps_force=[]

lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)


for config in structures:
    struct = config.toatoms()
    atomic_charges=[]
    for a in struct.get_chemical_symbols():                          
        atomic_charges.append(q[a])
    struct.set_initial_charges(atomic_charges)
    struct.calc = lammps   
    lammps_ene.append(struct.get_potential_energy())
    lammps_force.append(np.ravel(struct.get_forces()))
    
    
print(" Ewald energy from PYMATGEN:{}".format(pymat_ene))
print(" Ewald energy from LAMMPS:{}".format(lammps_ene))
print("MSE on ewald energies between PYMATGEN and LAMMPS: {}".format(mean_squared_error(np.array(lammps_ene),np.array(pymat_ene))))
print("MSE on ewald forces between PYMATGEN and LAMMPS:  {}".format(mean_squared_error(np.array(lammps_force),np.array(pymat_force))))


 Ewald energy from PYMATGEN:[-8170.758011429012, -8184.595515688825, -8186.09997270271, -8188.444604228178, -8213.698503604448]
 Ewald energy from LAMMPS:[-8170.760324043839, -8184.59774675683, -8186.102158114385, -8188.446804101816, -8213.70067096901]
MSE on ewald energies between PYMATGEN and LAMMPS: 4.927757828389627e-06
MSE on ewald forces between PYMATGEN and LAMMPS:  7.653966845525828e-11


# 2. Fit CCS+Q potential to trainset


In [4]:
input ={
        "General": {
                "interface": "CCS+Q"
        },
        "Twobody": {
                "O-Zn": {
                        "Rcut": 6.0,
                        "Resolution": 0.05,
						"Swtype":"sw"
                },
				"O-O": {
                        "Rcut": 6.0,
                        "Resolution": 0.1,
						"Swtype":"rep"
                },
				"Zn-Zn": {
                        "Rcut": 6.0,
                        "Resolution": 0.1,
						"Swtype":"rep"
                }
        }
}

#SAVE TO FILE
with open('CCS_input.json', 'w') as f:
    json.dump(input, f, indent=8)

#RUN FIT
ccs_fit("CCS_input.json")

    Generating one-body information from training-set.
        Added elements:  ['O', 'Zn']
    Applying monotonous constraints for pair:  O-Zn
    Applying monotonous constraints for pair:  O-O
    Applying monotonous constraints for pair:  Zn-Zn
    There is linear dependence in stochiometry matrix!
    Removing onebody term: Zn


    Finding optimum switch: 100%|███████████████████████████████| 90/90 [00:01<00:00, 47.58it/s]


    The best switch is (33, 36, 37) wtih mse: 4.7749e-11 
    Final root mean square error in fit:  1.9194553239849493e-08  (eV/atoms) [NOTE: Only elements specified in Onebody are considered!]


In [5]:
# Generate spline table for LAMMPS from json file
tags=asecalcTotable("CCS_params.json",scale=10)  # Controls the resolution of the gridsize; gridsize=dr/scale.
tags

{'O-Zn': {'Rmin': 1.5489944133954232,
  'Rcut': 6.048994413395423,
  'dr': 0.005,
  'N': 901},
 'O-O': {'Rmin': 2.50061211721814,
  'Rcut': 6.0006121172181395,
  'dr': 0.01,
  'N': 351},
 'Zn-Zn': {'Rmin': 2.4982061142105443,
  'Rcut': 6.098206114210544,
  'dr': 0.01,
  'N': 361}}

# 3.Define the calculators

In [10]:
# Optimize structure using CCS+Q ase caclulator

charges={"Zn":2.0,"O":-2.0}     # intial charges

with open ('CCS_params.json','r') as f:
    CCS_params=json.load(f)

CCS_params['One_body']['O']=0.0                           # Explicitly set the reference energies to 0.0
CCS_params['One_body']['Zn']=0.0 

#  CCS calculator
ccs= CCS(CCS_params,charge=True,q=charges,charge_scaling=True)

#  lAMMPS calculator
scaled_q={"Zn":2.0*CCS_params["Charge scaling factor"],"O":-2.0*CCS_params["Charge scaling factor"]}

parameters = {
              'atom_style': 'charge',
              'pair_style': 'hybrid/overlay table spline 1000  ewald coul/long 5.99',   
              'pair_coeff': ['* * coul/long',
                             '1 1  table CCS.table O-O 5.99',
                             '1 2  table CCS.table O-Zn 5.99',
                             '2 2  table CCS.table Zn-Zn 5.99'],
              'kspace_style': 'ewald 0.000001',
              'command': '/usr/bin/lmp'
                
}
lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)





# 4. Single point calculation

In [11]:
struct=read('POSCAR')
struct_ccs=copy.deepcopy(struct)

struct_ccs.calc=ccs
struct.calc=lammps

# Set charges 
atomic_charges=[]
for a in struct.get_chemical_symbols():                          
    atomic_charges.append(scaled_q[a])
    
struct.set_initial_charges(atomic_charges)

print("Energy from LAMMPS  pre optmization: ", struct.get_potential_energy())
print("Energy from CCS calculator pre optmization:", struct_ccs.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ccs.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ccs.get_forces())))


Energy from LAMMPS  pre optmization:  -202.2431902666347
Energy from CCS calculator pre optmization: -202.2430779132522
MSE on energy between ase calc and LAMMPS: 1.2623282563541405e-08
MSE on forces between ase calc and LAMMPS: 2.834896189130992e-11



# 5. Optimization


In [12]:
traj = Trajectory('LAMMPS.traj', 'w', struct_ccs)
dyn=BFGS(struct_ccs)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from CCS calculator after optimization: ", struct_ccs.get_potential_energy())

traj = Trajectory('CCS.traj', 'w', struct)
dyn=BFGS(struct)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from LAMMPS calculator after optimization:", struct.get_potential_energy())

print ("MSE on energy between CCS and LAMMPS calculator: {}".format((struct_ccs.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between CCS and LAMMPS calculator: {}".format(mean_squared_error(struct.get_forces(),struct_ccs.get_forces())))


      Step     Time          Energy         fmax
BFGS:    0 10:48:06     -202.243078        0.5762
BFGS:    1 10:48:06     -202.269565        0.4659
BFGS:    2 10:48:06     -202.319054        0.1869

BFGS:    3 10:48:07     -202.323075        0.1864
BFGS:    4 10:48:07     -202.337968        0.1295
BFGS:    5 10:48:07     -202.340687        0.1131
BFGS:    6 10:48:07     -202.346292        0.0896
BFGS:    7 10:48:07     -202.348333        0.0662
BFGS:    8 10:48:07     -202.349195        0.0414
Energy from CCS calculator after optimization:  -202.34919482303442
      Step     Time          Energy         fmax
BFGS:    0 10:48:07     -202.243190        0.5761
BFGS:    1 10:48:07     -202.269679        0.4659
BFGS:    2 10:48:07     -202.319166        0.1869
BFGS:    3 10:48:07     -202.323189        0.1864
BFGS:    4 10:48:07     -202.338086        0.1295
BFGS:    5 10:48:07     -202.340803        0.1131
BFGS:    6 10:48:07     -202.346414        0.0895
BFGS:    7 10:48:07     -202.3484

**Things to consider**:
* It is not possible to get exact same ewald energies in LAMMPS and PYMATGEN due to the differences in cut-offs and other hyper parameters used. 
* If LAMMPS needs to be used, its **advisable** that the training-set contains ewald energy contributions calculated from LAMMPS and not PYMATGEN. But, in certain cases the error might be negligible.
* **In LAMMPS, the real space cut-off for ewald and cut-off for short ranged spline potential should match.** 